In [ ]:
import sys
sys.path.append('..')
from config import tushare_api_key
import tushare as ts
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import sqlalchemy
import datetime
import pymysql

# 解决中文和负号显示问题
plt.rcParams['font.sans-serif'] = ['SimHei']  # 更改为SimHei字体以正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

print("设置Tushare令牌...")
ts.set_token(tushare_api_key)
pro = ts.pro_api()

# 数据库连接配置
database_config = {
    'host': '192.168.3.7',
    'user': 'stock',
    'password': 'Abcd1234!!',
    'database': 'aistock',
    'port': 3306
}

# 创建数据库连接
def create_db_connection(database_config):
    connection_string = f"mysql+pymysql://{database_config['user']}:{database_config['password']}@{database_config['host']}:{database_config['port']}/{database_config['database']}"
    engine = sqlalchemy.create_engine(connection_string)
    connection = engine.connect()
    return connection, engine

# 连接数据库
db_connection, db_engine = create_db_connection(database_config)
# 修改：直接使用engine来创建cursor对象
cursor = db_engine.raw_connection().cursor()

def calculate_macd(df, short_period=12, long_period=26, signal_period=9):
    df['EMA12'] = df['close'].ewm(span=short_period, adjust=False).mean()
    df['EMA26'] = df['close'].ewm(span=long_period, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['Signal_Line'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
    df['Volume'] = df['vol']  # 添加成交量
    return df

def plot_macd_with_golden_cross(stock_code):
    
    print(f"获取{stock_code}的每日历史报价数据...")
    # 使用今天往回1年的数据
    end_date = pd.Timestamp.today().strftime('%Y%m%d')
    start_date = (pd.Timestamp.today() - pd.Timedelta(days=365)).strftime('%Y%m%d')
    df = pro.daily(ts_code=stock_code, start_date=start_date, end_date=end_date)
    
    # 获取证券名称
    stock_info = pro.stock_basic(ts_code=stock_code, fields='name')
    stock_name = stock_info.iloc[0]['name']
    
    print("按日期排序数据框...")
    df_sorted = df.sort_values(by='trade_date')
    
    print("将trade_date转换为datetime格式...")
    df_sorted['trade_date'] = pd.to_datetime(df_sorted['trade_date'])
    
    print("计算MACD值和成交量...")
    df_macd = calculate_macd(df_sorted)
    
    print("绘制MACD图和成交量...")
    fig, ax1 = plt.subplots(figsize=(24, 3))  # 修改图的大小为1600x200
    ax2 = ax1.twinx()  # 创建第二个坐标轴
    ax1.plot(df_macd['trade_date'], df_macd['MACD'], label='MACD')
    ax1.plot(df_macd['trade_date'], df_macd['Signal_Line'], label='Signal Line')
    
    # 根据DIFF和DEA的位置调整成交量的颜色和位置
    for index, row in df_macd.iterrows():
        if row['MACD'] > row['Signal_Line']:
            color = 'red'
            ax2.bar(row['trade_date'], row['Volume'], color=color, alpha=0.3)
        else:
            color = 'green'
            ax2.bar(row['trade_date'], -row['Volume'], color=color, alpha=0.3)
    
    # 确保x轴上下方空间相等
    max_volume = df_macd['Volume'].max()
    ax2.set_ylim(-max_volume * 1.1, max_volume * 1.1)  # 设置y轴的最大值和最小值，使其绝对值相等并适当放大
    
    # 对MACD指标的坐标轴也做同样调整
    max_macd = max(df_macd['MACD'].max(), abs(df_macd['MACD'].min()))
    ax1.set_ylim(-max_macd * 1.1, max_macd * 1.1)  # 设置MACD y轴的最大值和最小值，使其绝对值相等并适当放大
    
    print("突出显示金叉点和死叉点...")
    # 修改金叉的计算逻辑，确保只有当MACD从下方穿越Signal Line时才被视为金叉
    golden_crosses = df_macd[(df_macd['MACD'] > df_macd['Signal_Line']) & (df_macd['MACD'].shift(1) < df_macd['Signal_Line'].shift(1))]
    
    # 新增死叉的计算逻辑，确保只有当MACD从上方穿越Signal Line时才被视为死叉
    death_crosses = df_macd[(df_macd['MACD'] < df_macd['Signal_Line']) & (df_macd['MACD'].shift(1) > df_macd['Signal_Line'].shift(1))]
    
    # 初始化变量以存储最近一次0线下方金叉的日期和偏移值
    last_gc_below_0_date = None
    gcbz_offset = None
    
    # 修改last_gc_below_0_date的定义为0线下方最后一个“叉”，且这个叉是金叉，这个叉后面没有别的叉了（尤其不能有死叉）
    crosses_below_0 = pd.concat([golden_crosses, death_crosses]).sort_index()
    crosses_below_0 = crosses_below_0[crosses_below_0['MACD'] < 0]
    if not crosses_below_0.empty:
        last_cross_below_0 = crosses_below_0.iloc[-1]
        if last_cross_below_0['MACD'] > last_cross_below_0['Signal_Line']:  # 确保最后一个叉是金叉
            last_gc_below_0_date = last_cross_below_0['trade_date']
            gcbz_offset = last_cross_below_0['MACD']
    
    for index, row in golden_crosses.iterrows():
        ax1.annotate('GC', xy=(mdates.date2num(row['trade_date']), min(row['MACD'], row['Signal_Line'])), 
                     xytext=(mdates.date2num(row['trade_date']), min(row['MACD'], row['Signal_Line'])-0.1 * 1.1),  # 尾巴长度调整为不超过10个像素并适当放大
                     arrowprops=dict(facecolor='red', shrink=0.05, headlength=5, headwidth=5, width=1))
    
    for index, row in death_crosses.iterrows():
        ax1.annotate('DC', xy=(mdates.date2num(row['trade_date']), max(row['MACD'], row['Signal_Line'])), 
                     xytext=(mdates.date2num(row['trade_date']), max(row['MACD'], row['Signal_Line'])+0.1 * 1.1),  # 尾巴长度调整为不超过10个像素并适当放大
                     arrowprops=dict(facecolor='green', shrink=0.05, headlength=5, headwidth=5, width=1))

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.title(f"{stock_code} {stock_name} - MACD图与金叉、死叉及成交量")  # 在标题中加入证券名称
    ax1.set_xlabel('日期')
    ax1.set_ylabel('MACD值')
    ax2.set_ylabel('成交量')
    plt.show()
    
    return last_gc_below_0_date, gcbz_offset

# 遍历深沪两市所有“0”和“6”开头的股票
# 使用tushare获取股票列表
stock_list = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code').query("ts_code.str.startswith('0') or ts_code.str.startswith('6')", engine='python')
# 提出3开头和01开头的品种
stock_list = stock_list[~stock_list['ts_code'].str.startswith(('3', '01'))]

import msvcrt  # Importing module for keypress detection

for index, row in stock_list.iterrows():
    stock_code = row['ts_code']
    # 从tushare获取股票的基本信息以解决stock_name未定义的问题
    stock_info = pro.stock_basic(ts_code=stock_code, fields='name')
    stock_name = stock_info['name'].iloc[0]
    # 检查今天是否已经测算过
    today = datetime.date.today()
    cursor.execute(f"SELECT * FROM goldencross WHERE stockCode='{stock_code}' AND miningDate='{today}'")
    if cursor.fetchone():
        print(f"{stock_code} 今天已经测算过了")
        continue

    # 调用类似plot_macd_with_golden_cross的函数计算金叉
    last_gc_below_0_date, gcbz_offset = plot_macd_with_golden_cross(stock_code)
    
    # 如果1年内数据中没有找到金叉
    if last_gc_below_0_date is None and gcbz_offset is None:
        cursor.execute(f"INSERT INTO goldencross (miningDate, stockCode, stockName, lastGCbelow0date, gcbzOffset) VALUES ('{today}', '{stock_code}', '{stock_name}', NULL, NULL)")
    else:
        cursor.execute(f"INSERT INTO goldencross (miningDate, stockCode, stockName, lastGCbelow0date, gcbzOffset) VALUES ('{today}', '{stock_code}', '{stock_name}', '{last_gc_below_0_date}', {gcbz_offset})")
    
    db_connection.commit()
    print(f"{stock_code} 的金叉数据已写入数据库")

    # Check for keypress after each stock calculation
    if msvcrt.kbhit():
        if msvcrt.getch() == b'q':  # If 'q' is pressed
            print("用户已请求中断运行。")
            break  # Exit the loop

# 关闭数据库连接
db_connection.close()

